Basemodel: DistilBERT

In [1]:
!git clone https://github.com/drryodino246/csc542-gp-team164.git

Cloning into 'csc542-gp-team164'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 13), reused 11 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 665.58 KiB | 8.42 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
import pandas as pd
emotions = pd.read_csv('/content/csc542-gp-team164/baseline.csv')

In [3]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(emotions, test_size=0.2, random_state=42)

In [28]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

emotions_encoded = emotions.map(tokenize, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

TypeError: tokenize() got an unexpected keyword argument 'batched'